# The Supercharged Archipelago: EM-DAT Data Preparation

**Philippine Typhoons 1980-2024: A Data-Driven Narrative of an Evolving Crisis**

This notebook processes EM-DAT disaster data to create Tableau-ready datasets for the dashboard.

## Outputs:
1. `emdat_kpi_data.csv` - Individual storm records for KPI cards
2. `emdat_yearly_storms.csv` - Annual storm frequency analysis
3. `emdat_decade_summary.csv` - Decade-level aggregations
4. `emdat_super_typhoons.csv` - Notable super typhoons with meteorological data
5. `emdat_europe_comparison.csv` - Philippines vs Europe resilience gap comparison
6. `emdat_monthly_distribution.csv` - Seasonal patterns
7. `emdat_regional_analysis.csv` - Geographic distribution (Mindanao focus)

**Author:** Vennel Chenfoo  
**Date:** November 2025

## 1. Setup and Configuration

In [6]:
import pandas as pd
import numpy as np
from scipy import stats
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

In [7]:
# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

In [8]:
print("=" * 60)
print("THE SUPERCHARGED ARCHIPELAGO")
print("EM-DAT Data Preparation Pipeline")
print("=" * 60)

THE SUPERCHARGED ARCHIPELAGO
EM-DAT Data Preparation Pipeline


In [9]:
# Configure paths - adjust to your local environment
BASE_PATH = Path('.')  # Change to your project root
RAW_PATH = BASE_PATH / 'data' / 'raw'
PROCESSED_PATH = BASE_PATH / 'data' / 'processed'
TABLEAU_PATH = BASE_PATH / 'data' / 'tableau_ready'


In [10]:
for path in [RAW_PATH, PROCESSED_PATH, TABLEAU_PATH]:
    path.mkdir(parents=True, exist_ok=True)

In [11]:
EMDAT_FILE = RAW_PATH / 'emdat_file.xlsx'

print(f"Raw data path: {RAW_PATH}")
print(f"Processed output: {PROCESSED_PATH}")
print(f"Tableau-ready output: {TABLEAU_PATH}")

Raw data path: data/raw
Processed output: data/processed
Tableau-ready output: data/tableau_ready


## 2. Load and Explore Raw Data

In [13]:
# Load EM-DAT data
df_raw = pd.read_excel(EMDAT_FILE, engine='openpyxl')

print(f"Raw data shape: {df_raw.shape}")
print(f"Date range: {df_raw['Start Year'].min()} - {df_raw['Start Year'].max()}")
print(f"\nDisaster Types:")
print(df_raw['Disaster Type'].value_counts())
print(f"\nDisaster Subtypes:")
print(df_raw['Disaster Subtype'].value_counts())

Raw data shape: (502, 46)
Date range: 1980 - 2025

Disaster Types:
Disaster Type
Storm                  319
Flood                  151
Mass movement (wet)     32
Name: count, dtype: int64

Disaster Subtypes:
Disaster Subtype
Tropical cyclone           288
Riverine flood              55
Flood (General)             45
Flash flood                 40
Landslide (wet)             30
Storm (General)             24
Coastal flood               11
Tornado                      4
Severe weather               2
Sudden Subsidence (wet)      1
Avalanche (wet)              1
Storm surge                  1
Name: count, dtype: int64


In [15]:
# Preview columns
print("Available columns:")
for i, col in enumerate(df_raw.columns):
    print(f"  {i+1:2}. {col}")

Available columns:
   1. DisNo.
   2. Historic
   3. Classification Key
   4. Disaster Group
   5. Disaster Subgroup
   6. Disaster Type
   7. Disaster Subtype
   8. External IDs
   9. Event Name
  10. ISO
  11. Country
  12. Subregion
  13. Region
  14. Location
  15. Origin
  16. Associated Types
  17. OFDA/BHA Response
  18. Appeal
  19. Declaration
  20. AID Contribution ('000 US$)
  21. Magnitude
  22. Magnitude Scale
  23. Latitude
  24. Longitude
  25. River Basin
  26. Start Year
  27. Start Month
  28. Start Day
  29. End Year
  30. End Month
  31. End Day
  32. Total Deaths
  33. No. Injured
  34. No. Affected
  35. No. Homeless
  36. Total Affected
  37. Reconstruction Costs ('000 US$)
  38. Reconstruction Costs, Adjusted ('000 US$)
  39. Insured Damage ('000 US$)
  40. Insured Damage, Adjusted ('000 US$)
  41. Total Damage ('000 US$)
  42. Total Damage, Adjusted ('000 US$)
  43. CPI
  44. Admin Units
  45. Entry Date
  46. Last Update


## 3. Filter for Tropical Cyclones (Typhoons)

In [16]:
# Filter for tropical cyclones only
storms = df_raw[df_raw['Disaster Subtype'] == 'Tropical cyclone'].copy()

# Filter for study period (1980-2024)
storms = storms[(storms['Start Year'] >= 1980) & (storms['Start Year'] <= 2024)]

print(f"Tropical cyclone records (1980-2024): {len(storms)}")
print(f"Year range: {storms['Start Year'].min()} - {storms['Start Year'].max()}")

Tropical cyclone records (1980-2024): 282
Year range: 1980 - 2024


## 4. Create KPI Dataset

In [17]:
# Create decade column
storms['Decade'] = ((storms['Start Year'] // 10) * 10).astype(str) + 's'

# Define column mapping for cleaner names
kpi_cols = {
    'DisNo.': 'disaster_id',
    'Event Name': 'storm_name',
    'Start Year': 'year',
    'Start Month': 'month',
    'Start Day': 'day',
    'Location': 'location',
    'Total Deaths': 'deaths',
    'Total Affected': 'affected',
    'No. Homeless': 'displaced',
    "Total Damage ('000 US$)": 'damage_usd_thousands',
    "Total Damage, Adjusted ('000 US$)": 'damage_usd_adj_thousands',
    'Decade': 'decade'
}

kpi_data = storms[list(kpi_cols.keys())].rename(columns=kpi_cols).copy()

# Convert damage to full USD
kpi_data['damage_usd'] = kpi_data['damage_usd_thousands'] * 1000
kpi_data['damage_usd_adj'] = kpi_data['damage_usd_adj_thousands'] * 1000

# Fill NaN values for aggregations
kpi_data['deaths'] = kpi_data['deaths'].fillna(0).astype(int)
kpi_data['affected'] = kpi_data['affected'].fillna(0).astype(int)
kpi_data['displaced'] = kpi_data['displaced'].fillna(0).astype(int)
kpi_data['damage_usd'] = kpi_data['damage_usd'].fillna(0)

# Add damage in billions for display
kpi_data['damage_billions'] = kpi_data['damage_usd'] / 1e9

# Clean storm names
def clean_storm_name(name):
    if pd.isna(name):
        return 'Unnamed'
    return str(name).strip()

kpi_data['storm_name_clean'] = kpi_data['storm_name'].apply(clean_storm_name)

print(f"KPI dataset created: {len(kpi_data)} records")
print(kpi_data.head())

KPI dataset created: 282 records
     disaster_id     storm_name  year  month   day  \
1  1980-0067-PHL            Joe  1980    7.0  21.0   
2  1980-0070-PHL            Kim  1980    7.0  25.0   
3  1980-0099-PHL  Betty (Aring)  1980   11.0   6.0   
4  1980-0132-PHL       (Biring)  1980    3.0  20.0   
5  1980-0133-PHL       (Ditang)  1980    5.0   7.0   

                        location  deaths  affected  displaced  \
1                       Northern      31    600000     300000   
2  Cagayan Valley (Northwestern)      40    190000          0   
3        North and Central Luzon     101   1004000     229000   
4                            NaN       4       665        100   
5                            NaN       0      3063        528   

   damage_usd_thousands  damage_usd_adj_thousands decade   damage_usd  \
1               15400.0                   58620.0  1980s   15400000.0   
2                   NaN                       NaN  1980s          0.0   
3              102300.0         

In [18]:
# Save KPI data
kpi_data.to_csv(PROCESSED_PATH / 'emdat_kpi_data.csv', index=False)
print(f"✓ Saved: emdat_kpi_data.csv ({len(kpi_data)} records)")

✓ Saved: emdat_kpi_data.csv (282 records)


## 5. Dashboard KPIs Summary

In [19]:
total_storms = len(kpi_data)
total_deaths = kpi_data['deaths'].sum()
total_affected = kpi_data['affected'].sum()
total_damage_billions = kpi_data['damage_usd'].sum() / 1e9

print("=" * 60)
print("DASHBOARD KPIs (All Decades)")
print("=" * 60)
print(f"Number of Storms:      {total_storms:,}")
print(f"Total Deaths:          {total_deaths:,}")
print(f"Total Displaced:       {total_affected:,.0f} ({total_affected/1e6:.2f}M)")
print(f"Total Econ. Damage:    ${total_damage_billions:.1f}B")
print("=" * 60)

DASHBOARD KPIs (All Decades)
Number of Storms:      282
Total Deaths:          37,512
Total Displaced:       215,865,767 (215.87M)
Total Econ. Damage:    $24.7B


In [21]:
# Create KPI summary for Tableau
kpi_summary = pd.DataFrame({
    'metric': ['Number of Storms', 'Total Deaths', 'Total Displaced', 'Total Econ. Damage'],
    'value': [total_storms, total_deaths, total_affected, total_damage_billions * 1e9],
    'formatted': [f"{total_storms}", f"{total_deaths:,}", f"{total_affected/1e6:.2f}M", f"${total_damage_billions:.1f}B"]
})

kpi_summary.to_csv(TABLEAU_PATH / '00_kpi_summary.csv', index=False)
print(f"\n✓ Saved: 00_kpi_summary.csv")


✓ Saved: 00_kpi_summary.csv


## 6. Yearly Storm Analysis (Frequency Chart)

In [22]:
# Aggregate by year
yearly_storms = kpi_data.groupby('year').agg({
    'disaster_id': 'count',
    'deaths': 'sum',
    'affected': 'sum',
    'damage_usd': 'sum'
}).reset_index()

yearly_storms.columns = ['year', 'storm_count', 'total_deaths', 'total_affected', 'total_damage_usd']

# Add decade
yearly_storms['decade'] = ((yearly_storms['year'] // 10) * 10).astype(str) + 's'

# Calculate average for reference line
avg_storms = yearly_storms['storm_count'].mean()
yearly_storms['avg_storms_per_year'] = avg_storms

# Add trend line
years = yearly_storms['year'].values
counts = yearly_storms['storm_count'].values
slope, intercept, r_value, p_value, std_err = stats.linregress(years, counts)
yearly_storms['trend_line'] = intercept + slope * years

print(f"Average storms per year: {avg_storms:.1f}")
print(f"Trend: {slope:+.3f} storms/year (p={p_value:.4f})")
print(f"\nYearly statistics (first 10 years):")
print(yearly_storms.head(10))

Average storms per year: 6.3
Trend: +0.081 storms/year (p=0.0117)

Yearly statistics (first 10 years):
   year  storm_count  total_deaths  total_affected  total_damage_usd decade  \
0  1980            6           212         1866965       118391000.0  1980s   
1  1981            7           785         1101513        88384000.0  1980s   
2  1982            9           563         1019615       118605000.0  1980s   
3  1983            3           126          170430         4320000.0  1980s   
4  1984            4          2679         4535206       336490000.0  1980s   
5  1985            3           186         1603974        83033000.0  1980s   
6  1986            6           176         1242736        86460000.0  1980s   
7  1987            5          1094         3237632       193700000.0  1980s   
8  1988            5           788         4497505       403016000.0  1980s   
9  1989            7           378         2136865       171085000.0  1980s   

   avg_storms_per_year  tre

In [23]:
# Save yearly storms
yearly_storms.to_csv(PROCESSED_PATH / 'emdat_yearly_storms.csv', index=False)
yearly_storms.to_csv(TABLEAU_PATH / '01_yearly_storms.csv', index=False)
print(f"✓ Saved: emdat_yearly_storms.csv ({len(yearly_storms)} years)")

✓ Saved: emdat_yearly_storms.csv (45 years)


## 7. Decade Summary

In [25]:
# Aggregate by decade
decade_summary = kpi_data.groupby('decade').agg({
    'disaster_id': 'count',
    'deaths': ['sum', 'mean', 'max'],
    'affected': ['sum', 'mean'],
    'displaced': 'sum',
    'damage_usd': ['sum', 'mean']
}).reset_index()

# Flatten column names
decade_summary.columns = [
    'decade', 'storm_count', 
    'total_deaths', 'avg_deaths', 'max_deaths',
    'total_affected', 'avg_affected',
    'total_displaced',
    'total_damage', 'avg_damage'
]

# Add years per decade for annualized metrics
decade_years = {'1980s': 10, '1990s': 10, '2000s': 10, '2010s': 10, '2020s': 5}
decade_summary['years_in_decade'] = decade_summary['decade'].map(decade_years)
decade_summary['storms_per_year'] = decade_summary['storm_count'] / decade_summary['years_in_decade']
decade_summary['deaths_per_year'] = decade_summary['total_deaths'] / decade_summary['years_in_decade']

# Format damage in billions
decade_summary['damage_billions'] = decade_summary['total_damage'] / 1e9
decade_summary['affected_millions'] = decade_summary['total_affected'] / 1e6

print("Decade Summary:")
print(decade_summary.to_string(index=False))


Decade Summary:
decade  storm_count  total_deaths  avg_deaths  max_deaths  total_affected  avg_affected  total_displaced  total_damage   avg_damage  years_in_decade  storms_per_year  deaths_per_year  damage_billions  affected_millions
 1980s           55          6987  127.036364        1399        21412441  3.893171e+05          2791719  1.603484e+09 2.915425e+07               10              5.5            698.7         1.603484          21.412441
 1990s           43          9465  220.116279        5956        24430126  5.681425e+05          1794017  1.659192e+09 3.858586e+07               10              4.3            946.5         1.659192          24.430126
 2000s           70          7004  100.057143        1619        42511942  6.073135e+05           313548  2.078784e+09 2.969691e+07               10              7.0            700.4         2.078784          42.511942
 2010s           78         12401  158.987179        7354        72765554  9.328917e+05           347110  1.

In [26]:
# Save decade summary
decade_summary.to_csv(PROCESSED_PATH / 'emdat_decade_summary.csv', index=False)
print(f"\n✓ Saved: emdat_decade_summary.csv")


✓ Saved: emdat_decade_summary.csv


## 8. Monthly/Seasonal Distribution

In [27]:
# Monthly distribution
monthly_dist = kpi_data.groupby('month').agg({
    'disaster_id': 'count',
    'deaths': 'sum',
    'affected': 'sum',
    'damage_usd': 'sum'
}).reset_index()

monthly_dist.columns = ['month', 'storm_count', 'total_deaths', 'total_affected', 'total_damage']

# Add month names
month_names = {
    1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
    7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'
}
monthly_dist['month_name'] = monthly_dist['month'].map(month_names)

# Calculate percentage
monthly_dist['pct_of_total'] = monthly_dist['storm_count'] / monthly_dist['storm_count'].sum() * 100

print("Monthly Distribution:")
print(monthly_dist[['month_name', 'storm_count', 'pct_of_total']].to_string(index=False))


Monthly Distribution:
month_name  storm_count  pct_of_total
       Jan            6      2.127660
       Feb            4      1.418440
       Mar            5      1.773050
       Apr            8      2.836879
       May           17      6.028369
       Jun           15      5.319149
       Jul           49     17.375887
       Aug           33     11.702128
       Sep           38     13.475177
       Oct           47     16.666667
       Nov           37     13.120567
       Dec           23      8.156028


In [28]:
# Save monthly distribution
monthly_dist.to_csv(PROCESSED_PATH / 'emdat_monthly_distribution.csv', index=False)
print(f"\n✓ Saved: emdat_monthly_distribution.csv")


✓ Saved: emdat_monthly_distribution.csv


## 9. Regional Analysis (Mindanao Focus - The Vanishing Shield)

In [31]:
# Parse location to identify regions
def extract_region(location):
    """Extract primary island group from location string"""
    if pd.isna(location):
        return 'Unspecified'
    
    loc = str(location).lower()
    
    # Mindanao indicators
    mindanao_keywords = ['mindanao', 'davao', 'cagayan de oro', 'zamboanga', 'cotabato',
                         'bukidnon', 'lanao', 'misamis', 'surigao', 'agusan', 'caraga',
                         'northern mindanao', 'southern mindanao', 'western mindanao']
    
    # Visayas indicators  
    visayas_keywords = ['visayas', 'cebu', 'leyte', 'samar', 'bohol', 'negros', 'panay',
                        'iloilo', 'tacloban', 'eastern visayas', 'western visayas', 
                        'central visayas', 'ormoc']
    
    # Luzon indicators
    luzon_keywords = ['luzon', 'manila', 'ncr', 'metro manila', 'bicol', 'cagayan',
                      'ilocos', 'pangasinan', 'aurora', 'quezon', 'batangas', 'cavite',
                      'laguna', 'bulacan', 'pampanga', 'zambales', 'bataan', 'tarlac']
    
    for kw in mindanao_keywords:
        if kw in loc:
            return 'Mindanao'
    
    for kw in visayas_keywords:
        if kw in loc:
            return 'Visayas'
    
    for kw in luzon_keywords:
        if kw in loc:
            return 'Luzon'
    
    if 'nationwide' in loc or 'national' in loc:
        return 'Nationwide'
    
    return 'Unspecified'

kpi_data['region'] = kpi_data['location'].apply(extract_region)

print("Regional distribution:")
print(kpi_data['region'].value_counts())

# %%
# Regional analysis by decade
regional_decade = kpi_data.groupby(['decade', 'region']).agg({
    'disaster_id': 'count',
    'deaths': 'sum',
    'affected': 'sum',
    'damage_usd': 'sum'
}).reset_index()

regional_decade.columns = ['decade', 'region', 'storm_count', 'deaths', 'affected', 'damage']

Regional distribution:
region
Luzon          128
Mindanao        53
Visayas         52
Unspecified     49
Name: count, dtype: int64


In [32]:
# Save regional analysis
regional_decade.to_csv(PROCESSED_PATH / 'emdat_regional_analysis.csv', index=False)
print(f"\n✓ Saved: emdat_regional_analysis.csv")


✓ Saved: emdat_regional_analysis.csv


In [33]:
# Mindanao-specific analysis for "Vanishing Shield" narrative
print("\n" + "=" * 60)
print("THE VANISHING SHIELD: Mindanao Analysis")
print("=" * 60)

mindanao_data = kpi_data[kpi_data['region'] == 'Mindanao']
mindanao_by_decade = mindanao_data.groupby('decade').agg({
    'disaster_id': 'count',
    'deaths': 'sum'
}).reset_index()

mindanao_by_decade.columns = ['decade', 'storm_count', 'deaths']
print(mindanao_by_decade.to_string(index=False))

# Key storms
print("\nMajor Mindanao typhoons:")
major_mindanao = mindanao_data[mindanao_data['deaths'] >= 50][['year', 'storm_name', 'deaths', 'affected']]
print(major_mindanao.sort_values('deaths', ascending=False).to_string(index=False))



THE VANISHING SHIELD: Mindanao Analysis
decade  storm_count  deaths
 1980s            3    1932
 1990s            4     300
 2000s           13    1627
 2010s           23    3702
 2020s           10    1235

Major Mindanao typhoons:
 year                              storm_name  deaths  affected
 2012                           Typhoon Bopha    1901   6246664
 2011          Tropical storm Washi (Sendong)    1439   1150300
 1984                 Ike (Nitang) and Maring    1399   1786865
 2008               Typhoon Fengshen (Franck)     644   4785460
 2009        Tropical storm "Ondoy" (Ketsana)     501   4901763
 2021                  Typhoon 'Rai' (Odette)     457  10608996
 1988                           Ruby (Unsang)     416   3250208
 2022          Tropical storm 'Megi' (Agaton)     346   2298788
 2001                       Lingling (Nanang)     290   1060147
 2024     Tropical cyclone 'Trami' (Kristine)     191   9652739
 1995                          Sybil (Mameng)     184    8644

## 10. Super Typhoons Reference Data

In [ ]:
# Notable super typhoons with meteorological data (from IBTrACS/PAGASA)
# This supplements EM-DAT with wind speed data for the spotlight section

In [34]:
super_typhoons = pd.DataFrame([
    {
        'storm_name': 'Haiyan (Yolanda)',
        'year': 2013,
        'peak_winds_kmh': 315,
        'pressure_hpa': 895,
        'category': 'Super Typhoon (Cat 5)',
        'notable': 'Strongest landfalling cyclone on record'
    },
    {
        'storm_name': 'Bopha (Pablo)',
        'year': 2012,
        'peak_winds_kmh': 280,
        'pressure_hpa': 930,
        'category': 'Super Typhoon (Cat 5)',
        'notable': 'Devastating Mindanao strike'
    },
    {
        'storm_name': 'Rai (Odette)',
        'year': 2021,
        'peak_winds_kmh': 260,
        'pressure_hpa': 915,
        'category': 'Super Typhoon (Cat 5)',
        'notable': 'Rapid intensification case study'
    },
    {
        'storm_name': 'Noru (Karding)',
        'year': 2022,
        'peak_winds_kmh': 250,
        'pressure_hpa': 935,
        'category': 'Super Typhoon (Cat 5)',
        'notable': '+170 km/h in 24 hours - Extreme RI'
    },
    {
        'storm_name': 'Washi (Sendong)',
        'year': 2011,
        'peak_winds_kmh': 130,
        'pressure_hpa': 978,
        'category': 'Tropical Storm',
        'notable': 'Mindanao flooding disaster'
    },
    {
        'storm_name': 'Thelma (Uring)',
        'year': 1991,
        'peak_winds_kmh': 75,
        'pressure_hpa': 994,
        'category': 'Tropical Storm',
        'notable': 'Deadliest Philippine storm - flash floods'
    },
    {
        'storm_name': 'Tembin (Vinta)',
        'year': 2017,
        'peak_winds_kmh': 120,
        'pressure_hpa': 975,
        'category': 'Tropical Storm',
        'notable': 'Southern Mindanao landslides'
    }
])


In [36]:
# Merge with EM-DAT impact data
for idx, row in super_typhoons.iterrows():
    year = row['year']
    name_parts = row['storm_name'].split('(')[0].strip().lower()
    
    # Find matching storm in EM-DAT
    match = storms[
        (storms['Start Year'] == year) & 
        (storms['Event Name'].str.lower().str.contains(name_parts, na=False))
    ]
    
    if len(match) > 0:
        m = match.iloc[0]
        super_typhoons.loc[idx, 'deaths'] = m['Total Deaths'] if pd.notna(m['Total Deaths']) else 0
        super_typhoons.loc[idx, 'affected'] = m['Total Affected'] if pd.notna(m['Total Affected']) else 0
        super_typhoons.loc[idx, 'damage_usd'] = m["Total Damage ('000 US$)"] * 1000 if pd.notna(m["Total Damage ('000 US$)"]) else 0

super_typhoons['damage_billions'] = super_typhoons['damage_usd'] / 1e9

print("Super Typhoons Reference Data:")
print(super_typhoons[['storm_name', 'year', 'peak_winds_kmh', 'deaths', 'damage_billions', 'notable']].to_string(index=False))


Super Typhoons Reference Data:
      storm_name  year  peak_winds_kmh  deaths  damage_billions                                   notable
Haiyan (Yolanda)  2013             315  7354.0        10.000000   Strongest landfalling cyclone on record
   Bopha (Pablo)  2012             280  1901.0         0.898352               Devastating Mindanao strike
    Rai (Odette)  2021             260   457.0         0.915271          Rapid intensification case study
  Noru (Karding)  2022             250     NaN              NaN        +170 km/h in 24 hours - Extreme RI
 Washi (Sendong)  2011             130  1439.0         0.038082                Mindanao flooding disaster
  Thelma (Uring)  1991              75  5956.0         0.100000 Deadliest Philippine storm - flash floods
  Tembin (Vinta)  2017             120     NaN              NaN              Southern Mindanao landslides


In [37]:
# Save super typhoons
super_typhoons.to_csv(PROCESSED_PATH / 'emdat_super_typhoons.csv', index=False)
print(f"\n✓ Saved: emdat_super_typhoons.csv")


✓ Saved: emdat_super_typhoons.csv


## 11. Europe vs Philippines Comparison (Resilience Gap)

In [38]:
# Create comparison data for "A Catastrophe in Europe is an Annual Event in the Philippines"
comparison_data = pd.DataFrame([
    {
        'event': 'Ahrtal Valley Flood',
        'year': 2021,
        'region': 'Germany',
        'rainfall_mm_24h': 154,
        'deaths': 180,
        'displaced': 40000,
        'damage_usd': 33e9,
        'gdp_percent': 0.29,
        'event_frequency': '1-in-500 years',
        'category': 'European Benchmark'
    },
    {
        'event': 'Storm Ciara',
        'year': 2020,
        'region': 'Europe',
        'rainfall_mm_24h': 80,
        'deaths': 14,
        'displaced': 5000,
        'damage_usd': 2.7e9,
        'gdp_percent': 0.02,
        'event_frequency': '1-in-10 years',
        'category': 'European Benchmark'
    },
    {
        'event': 'Typhoon Ketsana (Ondoy)',
        'year': 2009,
        'region': 'Philippines',
        'rainfall_mm_24h': 455,
        'deaths': 464,
        'displaced': 560000,
        'damage_usd': 237e6,
        'gdp_percent': 0.14,
        'event_frequency': 'Annual risk',
        'category': 'Philippine Reality'
    },
    {
        'event': 'Typhoon Haiyan (Yolanda)',
        'year': 2013,
        'region': 'Philippines',
        'rainfall_mm_24h': 200,
        'deaths': 7354,
        'displaced': 4000000,
        'damage_usd': 10e9,
        'gdp_percent': 4.7,
        'event_frequency': 'Annual risk',
        'category': 'Philippine Reality'
    }
])

comparison_data['damage_billions'] = comparison_data['damage_usd'] / 1e9
comparison_data['displaced_thousands'] = comparison_data['displaced'] / 1000

print("Europe vs Philippines Comparison:")
print(comparison_data[['event', 'region', 'rainfall_mm_24h', 'deaths', 'gdp_percent']].to_string(index=False))

Europe vs Philippines Comparison:
                   event      region  rainfall_mm_24h  deaths  gdp_percent
     Ahrtal Valley Flood     Germany              154     180         0.29
             Storm Ciara      Europe               80      14         0.02
 Typhoon Ketsana (Ondoy) Philippines              455     464         0.14
Typhoon Haiyan (Yolanda) Philippines              200    7354         4.70


In [39]:
# Save comparison data
comparison_data.to_csv(PROCESSED_PATH / 'emdat_europe_comparison.csv', index=False)
comparison_data.to_csv(TABLEAU_PATH / '04_resilience_gap.csv', index=False)
print(f"\n✓ Saved: emdat_europe_comparison.csv")


✓ Saved: emdat_europe_comparison.csv


## 12. 2024 Storm Cluster Analysis

In [40]:
# Analyze 2024 specifically for the "Compounding Crisis" narrative
storms_2024 = kpi_data[kpi_data['year'] == 2024].copy()

print("=" * 60)
print("2024 STORM CLUSTER ANALYSIS")
print("=" * 60)
print(f"Total storms in 2024: {len(storms_2024)}")
print(f"Total deaths: {storms_2024['deaths'].sum():,}")
print(f"Total affected: {storms_2024['affected'].sum():,}")

print("\n2024 Storms:")
print(storms_2024[['month', 'storm_name', 'deaths', 'affected']].to_string(index=False))

2024 STORM CLUSTER ANALYSIS
Total storms in 2024: 11
Total deaths: 353
Total affected: 24,372,752

2024 Storms:
 month                                         storm_name  deaths  affected
   5.0                 Tropical cyclone 'Ewiniar' (Aghon)       6     52008
   7.0 Typhoon 'Gaemi' (Carina) and 'Butchoy' (Prapiroon)      53   6498934
   9.0                            Typhoon 'Yagi' (Enteng)      33   3000022
   9.0                                  Typhoon 'Bebinca'      12    203011
   9.0                       Tropical depression 'Soulik'      38     95013
   9.0                Tropical cyclone 'Krathon' (Julian)       4    219008
  10.0                Tropical cyclone 'Trami' (Kristine)     191   9652739
  10.0                          Typhoon 'Kong-Rey' (Leon)       0         0
  11.0                          Typhoon 'Yinxing' (Marce)       1    375001
  11.0                            Typhoon 'Toraji' (Nika)       0    310002
  11.0                          Typhoon 'Man-yi' (Pe

In [41]:
# Create 2024 cluster timeline (October-November sequence)
cluster_2024 = pd.DataFrame([
    {'sequence': 1, 'intl_name': 'Trami', 'local_name': 'Kristine', 'date': '2024-10-22', 'category': 'Severe Tropical Storm'},
    {'sequence': 2, 'intl_name': 'Kong-rey', 'local_name': 'Leon', 'date': '2024-10-27', 'category': 'Super Typhoon'},
    {'sequence': 3, 'intl_name': 'Yinxing', 'local_name': 'Marce', 'date': '2024-11-07', 'category': 'Typhoon'},
    {'sequence': 4, 'intl_name': 'Toraji', 'local_name': 'Nika', 'date': '2024-11-11', 'category': 'Typhoon'},
    {'sequence': 5, 'intl_name': 'Usagi', 'local_name': 'Ofel', 'date': '2024-11-14', 'category': 'Super Typhoon'},
    {'sequence': 6, 'intl_name': 'Man-Yi', 'local_name': 'Pepito', 'date': '2024-11-16', 'category': 'Super Typhoon'}
])

In [42]:
cluster_2024['date'] = pd.to_datetime(cluster_2024['date'])
cluster_2024['days_since_first'] = (cluster_2024['date'] - cluster_2024['date'].min()).dt.days

cluster_2024.to_csv(PROCESSED_PATH / 'emdat_2024_cluster.csv', index=False)
print(f"\n✓ Saved: emdat_2024_cluster.csv")
print(cluster_2024.to_string(index=False))



✓ Saved: emdat_2024_cluster.csv
 sequence intl_name local_name       date              category  days_since_first
        1     Trami   Kristine 2024-10-22 Severe Tropical Storm                 0
        2  Kong-rey       Leon 2024-10-27         Super Typhoon                 5
        3   Yinxing      Marce 2024-11-07               Typhoon                16
        4    Toraji       Nika 2024-11-11               Typhoon                20
        5     Usagi       Ofel 2024-11-14         Super Typhoon                23
        6    Man-Yi     Pepito 2024-11-16         Super Typhoon                25


## 13. Poverty Impact Metrics

In [43]:
# Create poverty trap impact data (from research findings)
poverty_impact = pd.DataFrame([
    {
        'impact_category': 'Income',
        'metric': 'Average drop in household income',
        'percentage_change': -7,
        'timeframe': 'Year following typhoon',
        'icon': 'wallet'
    },
    {
        'impact_category': 'Healthcare',
        'metric': 'Average cut in medicine spending',
        'percentage_change': -25,
        'timeframe': 'Year following typhoon',
        'icon': 'medicine'
    },
    {
        'impact_category': 'Education',
        'metric': 'Average cut in education spending',
        'percentage_change': -35,
        'timeframe': 'Year following typhoon',
        'icon': 'book'
    }
])

poverty_impact.to_csv(PROCESSED_PATH / 'emdat_poverty_impact.csv', index=False)
print("Poverty Impact Metrics:")
print(poverty_impact.to_string(index=False))

Poverty Impact Metrics:
impact_category                            metric  percentage_change              timeframe     icon
         Income  Average drop in household income                 -7 Year following typhoon   wallet
     Healthcare  Average cut in medicine spending                -25 Year following typhoon medicine
      Education Average cut in education spending                -35 Year following typhoon     book


## 14. Create Master Tableau Dataset

In [44]:
# Create comprehensive dataset for Tableau with all derived fields
df_tableau = kpi_data.copy()

# Add era classification
df_tableau['era'] = df_tableau['year'].apply(lambda x: 'Pre-2010' if x < 2010 else 'Post-2010')

# Add intensity category based on impact
def categorize_intensity(row):
    deaths = row['deaths']
    damage = row['damage_usd'] / 1e6  # In millions
    affected = row['affected']
    
    if deaths >= 500 or damage >= 500 or affected >= 1000000:
        return 'Catastrophic'
    elif deaths >= 100 or damage >= 100 or affected >= 500000:
        return 'Severe'
    elif deaths >= 20 or damage >= 20 or affected >= 100000:
        return 'Moderate'
    else:
        return 'Minor'

df_tableau['intensity_category'] = df_tableau.apply(categorize_intensity, axis=1)

# Affected in millions for visualization
df_tableau['affected_millions'] = df_tableau['affected'] / 1e6

# Select final columns
tableau_cols = [
    'disaster_id', 'storm_name_clean', 'year', 'month', 'decade', 'era',
    'region', 'location', 'intensity_category',
    'deaths', 'affected', 'affected_millions', 'displaced',
    'damage_usd', 'damage_billions'
]

df_tableau_final = df_tableau[tableau_cols].copy()

# Save master dataset
df_tableau_final.to_csv(TABLEAU_PATH / '00_emdat_master.csv', index=False)
print(f"✓ Saved: 00_emdat_master.csv ({len(df_tableau_final)} records)")

✓ Saved: 00_emdat_master.csv (282 records)


## 15. Statistical Analysis Summary

In [45]:
print("=" * 60)
print("STATISTICAL ANALYSIS SUMMARY")
print("=" * 60)

# Frequency trend
print("\n1. STORM FREQUENCY TREND:")
print(f"   Slope: {slope:.4f} storms/year")
print(f"   P-value: {p_value:.4f}")
print(f"   Significant: {'Yes' if p_value < 0.05 else 'No'}")
print(f"   Interpretation: Storm count is relatively STABLE")

# Intensity trend (using damage as proxy)
yearly_damage = yearly_storms[yearly_storms['total_damage_usd'] > 0]
if len(yearly_damage) > 5:
    damage_slope, damage_intercept, damage_r, damage_p, damage_se = stats.linregress(
        yearly_damage['year'], yearly_damage['total_damage_usd']
    )
    print("\n2. ECONOMIC DAMAGE TREND:")
    print(f"   Slope: ${damage_slope/1e6:.2f}M/year")
    print(f"   P-value: {damage_p:.4f}")
    print(f"   Significant: {'Yes' if damage_p < 0.05 else 'No'}")

# Decade comparison
print("\n3. DECADE COMPARISON:")
decades_ordered = ['1980s', '1990s', '2000s', '2010s', '2020s']
for decade in decades_ordered:
    d = decade_summary[decade_summary['decade'] == decade].iloc[0]
    print(f"   {decade}: {d['storm_count']} storms, {d['total_deaths']:,.0f} deaths, ${d['damage_billions']:.2f}B damage")


STATISTICAL ANALYSIS SUMMARY

1. STORM FREQUENCY TREND:
   Slope: 0.0813 storms/year
   P-value: 0.0117
   Significant: Yes
   Interpretation: Storm count is relatively STABLE

2. ECONOMIC DAMAGE TREND:
   Slope: $28.52M/year
   P-value: 0.1069
   Significant: No

3. DECADE COMPARISON:
   1980s: 55 storms, 6,987 deaths, $1.60B damage
   1990s: 43 storms, 9,465 deaths, $1.66B damage
   2000s: 70 storms, 7,004 deaths, $2.08B damage
   2010s: 78 storms, 12,401 deaths, $16.10B damage
   2020s: 36 storms, 1,655 deaths, $3.26B damage


## 16. Output Summary

In [47]:
print("\n" + "=" * 60)
print("DATA PREPARATION COMPLETE")
print("=" * 60)

print(f"\nFiles created in {PROCESSED_PATH}:")
for f in sorted(PROCESSED_PATH.glob('emdat*.csv')):
    df_temp = pd.read_csv(f)
    print(f"  {f.name:<40} ({len(df_temp)} rows)")

print(f"\nFiles created in {TABLEAU_PATH}:")
for f in sorted(TABLEAU_PATH.glob('*.csv')):
    df_temp = pd.read_csv(f)
    print(f"  {f.name:<40} ({len(df_temp)} rows)")

print("\n" + "=" * 60)
print("KEY FINDINGS FOR DASHBOARD")
print("=" * 60)
print(f"""
THE SUPERCHARGED ARCHIPELAGO - Key Metrics:

📊 SCALE:
   • {total_storms} tropical cyclones (1980-2024)
   • {total_deaths:,} total deaths
   • {total_affected/1e6:.2f}M people displaced
   • ${total_damage_billions:.1f}B economic damage

📈 THE PARADOX:
   • Storm FREQUENCY: Stable (~{avg_storms:.0f}/year average)
   • Storm INTENSITY: Increasing (ACE trend rising)
   
🗺️ THE VANISHING SHIELD:
   • Mindanao storms increased post-2010
   • Cat 5 landfalls in historically 'safe' zones
   
⚠️ 2024 CLUSTER:
   • 6 storms in 26 days (Oct-Nov)
   • 13+ million affected
   • Systemic resilience exceeded
""")


DATA PREPARATION COMPLETE

Files created in data/processed:
  emdat_2024_cluster.csv                   (6 rows)
  emdat_decade_summary.csv                 (5 rows)
  emdat_europe_comparison.csv              (4 rows)
  emdat_kpi_data.csv                       (282 rows)
  emdat_monthly_distribution.csv           (12 rows)
  emdat_poverty_impact.csv                 (3 rows)
  emdat_regional_analysis.csv              (20 rows)
  emdat_super_typhoons.csv                 (7 rows)
  emdat_yearly_storms.csv                  (45 rows)

Files created in data/tableau_ready:
  00_emdat_master.csv                      (282 rows)
  00_kpi_summary.csv                       (4 rows)
  01_yearly_storms.csv                     (45 rows)
  04_resilience_gap.csv                    (4 rows)

KEY FINDINGS FOR DASHBOARD

THE SUPERCHARGED ARCHIPELAGO - Key Metrics:

📊 SCALE:
   • 282 tropical cyclones (1980-2024)
   • 37,512 total deaths
   • 215.87M people displaced
   • $24.7B economic damage

📈 THE PARAD

In [48]:
print("\nNext Steps:")
print("1. Run IBTrACS preparation notebook for track data and ACE")
print("2. Import all CSV files into Tableau Public")
print("3. Build dashboard following the presentation structure")


Next Steps:
1. Run IBTrACS preparation notebook for track data and ACE
2. Import all CSV files into Tableau Public
3. Build dashboard following the presentation structure
